# Evaluation CV-based Ball Tracking

In This model the CV model is evaluated on the test split with the hyperparameters identified in 1

### imports

In [4]:
import cv2
import numpy as np
import json
import pandas as pd
from tqdm import tqdm
import time

### Hyperparameters

In [5]:
# Hyperparameters
FRAME_WIDTH = 1280
FRAME_HEIGHT = 720
FPS = 30  
BG_DETECT_SHADOWS = True
FRAME_DIFF_THRESHOLD = 20
PLAYER_MIN_AREA = 300

# apply best config
BG_HISTORY = 300
BG_VAR_THRESHOLD = 48
DILATE_ITERATIONS = 3
ERODE_ITERATIONS = 1
BALL_MIN_AREA = 20
BALL_MAX_AREA = 150

# Parameters for tracking previous player positions
PREVIOUS_PLAYER_FRAMES = 10  # Number of frames to retain previous player positions
player_boxes_history = []  # Stores player bounding boxes from recent frames


### Functions

In [14]:
def scale_points(point, scale):
    # scale and round to int
    return (int(point[0] * scale), int(point[1] * scale))

def convert_json_to_df(json_data, split='train', resolution=[1280, 720], file_path='../../00_Dataset/'):
    rows = []
    for subset in json_data['subsets']:
        resolution_subset= subset['resolution']
        if 'ball' in subset['objects']:
            for video in subset['videos']:
                for clip in video['clips']:
                    for frame_number, frame in clip['frames_with_objects'].items():    
                        if frame['split'] != split:
                            continue
                        points = []
                        for ball in frame['balls']:
                            if ball['visibility'] not in ['Outside'] and ball['trajectory'] not in ['', 'Static']:
                                point = scale_points([ball['x'],ball['y']], resolution[0] / resolution_subset[0])
                                point_object = {'x': point[0], 'y': point[1], 'visibility': ball['visibility']}
                                points.append(point_object)

                            
                        file_path_full = file_path + subset['name'] + '/' + video['name'] + '/' + clip['name'] + '/' + frame_number + '.jpg'
                        rows.append({
                            'subset': subset['name'],
                            'video': video['name'],
                            'clip': clip['name'],
                            'frame': int(frame_number), 
                            'file_path': file_path_full, 
                            'points': points
                        })

    return pd.DataFrame(rows)

# get all frame sfrom a uniqe combination ordered by frame number
def get_frames(df, subset, video, clip):
    return df[(df['subset'] == subset) & (df['video'] == video) & (df['clip'] == clip)].sort_values('frame')

def initialize_background_subtractor():
    return cv2.createBackgroundSubtractorMOG2(
        history=BG_HISTORY,
        varThreshold=BG_VAR_THRESHOLD,
        detectShadows=BG_DETECT_SHADOWS
    )

def initialize_video_writer(output_path, frame_width, frame_height):
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    return cv2.VideoWriter(output_path, fourcc, FPS, (frame_width, frame_height))

def apply_background_subtraction(background_subtractor, frame):
    return background_subtractor.apply(frame)

def calculate_frame_difference(prev_frame, current_frame):
    frame_diff = cv2.absdiff(prev_frame, current_frame)
    gray_diff = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)
    _, diff_mask = cv2.threshold(gray_diff, FRAME_DIFF_THRESHOLD, 255, cv2.THRESH_BINARY)
    return diff_mask

def refine_mask(combined_mask):
    _, thresh = cv2.threshold(combined_mask, 200, 255, cv2.THRESH_BINARY)
    thresh = cv2.dilate(thresh, None, iterations=DILATE_ITERATIONS)
    thresh = cv2.erode(thresh, None, iterations=ERODE_ITERATIONS)
    return thresh

def combine_overlapping_boxes(boxes):
    merged_boxes = []

    # Continue to merge until no more overlaps are found
    while boxes:
        box = boxes.pop(0)
        x1, y1, w1, h1 = box
        has_merged = False

        # Check current box against merged boxes
        for i in range(len(merged_boxes)):
            x2, y2, w2, h2 = merged_boxes[i]
            
            # Check for overlap
            if (x1 < x2 + w2 and x1 + w1 > x2 and
                y1 < y2 + h2 and y1 + h1 > y2):
                
                # Merge the boxes into a new bounding box
                new_x = min(x1, x2)
                new_y = min(y1, y2)
                new_w = max(x1 + w1, x2 + w2) - new_x
                new_h = max(y1 + h1, y2 + h2) - new_y
                merged_boxes[i] = (new_x, new_y, new_w, new_h)  # Update merged box
                has_merged = True
                break

        # If box did not merge, add it as a new entry
        if not has_merged:
            merged_boxes.append(box)

    # Repeat merging process on merged_boxes list until stable
    stable = False
    while not stable:
        stable = True
        new_merged_boxes = []
        while merged_boxes:
            box = merged_boxes.pop(0)
            x1, y1, w1, h1 = box
            has_merged = False

            for i in range(len(new_merged_boxes)):
                x2, y2, w2, h2 = new_merged_boxes[i]
                
                # Check for overlap again
                if (x1 < x2 + w2 and x1 + w1 > x2 and
                    y1 < y2 + h2 and y1 + h1 > y2):
                    
                    # Merge overlapping boxes
                    new_x = min(x1, x2)
                    new_y = min(y1, y2)
                    new_w = max(x1 + w1, x2 + w2) - new_x
                    new_h = max(y1 + h1, y2 + h2) - new_y
                    new_merged_boxes[i] = (new_x, new_y, new_w, new_h)
                    has_merged = True
                    stable = False
                    break

            if not has_merged:
                new_merged_boxes.append(box)
        
        merged_boxes = new_merged_boxes

    return merged_boxes

def detect_objects(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Initialize lists to store detected areas for this frame
    circle_areas = []
    player_areas = []

    # Iterate over contours to classify as ball or player
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if BALL_MIN_AREA < area < BALL_MAX_AREA:  # Ball detection
            (x, y), radius = cv2.minEnclosingCircle(cnt)
            circle_areas.append((x, y, radius))  # Add to circle areas
        elif area > PLAYER_MIN_AREA:  # Player detection
            x, y, w, h = cv2.boundingRect(cnt)
            player_areas.append((x, y, w, h))  # Add to player areas

    # Combine overlapping player boxes
    player_areas = combine_overlapping_boxes(player_areas)

    # Add player areas to history for ball exclusion
    player_boxes_history.append(player_areas)
    if len(player_boxes_history) > PREVIOUS_PLAYER_FRAMES:
        player_boxes_history.pop(0)  # Limit history to the last 10 frames


    # Remove ball positions that overlap with player boxes from recent frames
    for players in player_boxes_history:
        for (x, y, w, h) in players:
             # Create a new list with only circles that don't overlap with the player box
            circle_areas = [
                (x1, y1, r) for (x1, y1, r) in circle_areas
                if not (x - r < x1 < x + w + r and y - r < y1 < y + h + r)
            ]

    for (x, y, w, h) in player_areas:
        for i, (x1, y1, r) in enumerate(circle_areas):
                 # Create a new list with only circles that don't overlap with the player box
                circle_areas = [
                    (x1, y1, r) for (x1, y1, r) in circle_areas
                    if not (x - r < x1 < x + w + r and y - r < y1 < y + h + r)
                ]

    ball_points = [] 
    # Draw valid ball detections
    for x, y, r in circle_areas:
        ball_points.append((int(x), int(y)))

    player_bbox = []
    # Draw player detections
    for x, y, w, h in player_areas:
        player_bbox.append((x, y, x + w, y + h))

    return ball_points, player_bbox

def process_image_sequence_just_annotations(df):
    global FRAME_WIDTH, FRAME_HEIGHT

    # Add required columns for annotations
    df['ball_points'] = None
    df['player_bbox'] = None

    # Initialize background subtractor
    background_subtractor = initialize_background_subtractor()

    # Read the first frame to initialize previous frame for differencing
    first_frame_path = df.iloc[0]['file_path']
    prev_frame = cv2.imread(first_frame_path)
    if prev_frame is None:
        print("Error: Could not read the first frame.")
        exit()

    # if resolution not 720 p rescale
    if prev_frame.shape[0] != FRAME_HEIGHT or prev_frame.shape[1] != FRAME_WIDTH:
        prev_frame = cv2.resize(prev_frame, (FRAME_WIDTH, FRAME_HEIGHT))

    # Iterate over DataFrame to process each frame
    for index, row in df.iterrows():
        frame_path = row['file_path']
        frame = cv2.imread(frame_path)
        if frame is None:
            print(f"Warning: Could not read frame at {frame_path}. Skipping.")
            continue

        # if resolution not 720 p rescale
        if frame.shape[0] != FRAME_HEIGHT or frame.shape[1] != FRAME_WIDTH:
            frame = cv2.resize(frame, (FRAME_WIDTH, FRAME_HEIGHT))

        # Remove noise
        frame = cv2.GaussianBlur(frame, (5, 5), 0)

        # Step 1: Background subtraction
        bg_mask = apply_background_subtraction(background_subtractor, frame)

        # Step 2: Frame differencing
        diff_mask = calculate_frame_difference(prev_frame, frame)
        prev_frame = frame  # Update previous frame

        # Step 3: Combine masks
        combined_mask = cv2.bitwise_and(bg_mask, diff_mask)

        # Step 4: Refine mask
        refined_mask = refine_mask(combined_mask)

        # Step 5: Detect objects
        ball_points, player_bbox = detect_objects(refined_mask)

        # add ball points to df
        df.at[index, 'ball_points'] = ball_points

        # add player bbox to df
        df.at[index, 'player_bbox'] = player_bbox

    return df

# def caluclate metrics 
def calculate_metrics_df(df, loose=True):
    metrics_dict = {'Dataset_Amateur': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Dataset_TrackNet': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Visibility_Visible': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Visibility_Difficult': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Visibility_Blurry': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Visibility_Occluded': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Visibility_None': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Total': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0}}
    
    previous_position = None

    for index, row in df.iterrows():
        gt_balls = row['points']
        pred_balls = row['ball_points']
        if loose:
            tp_frame, fp_frame, tn_frame, fn_frame = calculate_metrics_loose(gt_balls, pred_balls) 
        else:
            tp_frame, fp_frame, tn_frame, fn_frame, previous_position = calculate_metrics_strict(gt_balls, pred_balls, previous_position)

        dataset="Dataset_"+row['subset']
        visibility = "Visibility_" + (row['points'][0]['visibility'] if  len(row['points']) > 0 else 'None')

        metrics_dict[dataset]['tp'] += tp_frame   
        metrics_dict[dataset]['fp'] += fp_frame
        metrics_dict[dataset]['tn'] += tn_frame
        metrics_dict[dataset]['fn'] += fn_frame

        metrics_dict[visibility]['tp'] += tp_frame
        metrics_dict[visibility]['fp'] += fp_frame
        metrics_dict[visibility]['tn'] += tn_frame
        metrics_dict[visibility]['fn'] += fn_frame

        metrics_dict['Total']['tp'] += tp_frame
        metrics_dict['Total']['fp'] += fp_frame
        metrics_dict['Total']['tn'] += tn_frame
        metrics_dict['Total']['fn'] += fn_frame

    return metrics_dict


# calculate tp, fp, tn, fn
def calculate_metrics_strict(gt_balls, pred_balls, prev_position = None, distance_threshold=10):
    if len(gt_balls) == 0 and len(pred_balls) == 0:
        return 0, 0, 1, 0, None
    
    if len(gt_balls) == 0 and len(pred_balls) > 0:
        return 0, 1, 0, 0, pred_balls[0]
    
    if len(gt_balls) > 0 and len(pred_balls) == 0:
        return 0, 0, 0, 1, None
        
    pred_ball= None
    # if more than one preditcion get the closest one to prefious position
    if len(pred_balls) > 1 and prev_position is not None:
        distances = []
        for pred_ball in pred_balls:
            distance = np.linalg.norm(np.array(prev_position) - np.array(pred_ball))
            distances.append(distance)
        pred_ball = pred_balls[np.argmin(distances)]
    else:
        pred_ball = pred_balls[0]

    # get the first ball in the list
    gt_ball = (gt_balls[0]['x'], gt_balls[0]['y'])
    pred_ball = pred_balls[0]

    #calculate euclidean distance
    distance = np.linalg.norm(np.array(gt_ball) - np.array(pred_ball))

    if distance < distance_threshold:
        return 1, 0, 0, 0, pred_ball
    else:
        return 0, 1, 0, 0, pred_ball

    

# calculate tp, fp, tn, fn
def calculate_metrics_loose(gt_balls, pred_balls, distance_threshold=10):
    if len(gt_balls) == 0 and len(pred_balls) == 0:
        return 0, 0, 1, 0
    
    if len(gt_balls) == 0 and len(pred_balls) > 0:
        return 0, 1, 0, 0
    
    if len(gt_balls) > 0 and len(pred_balls) == 0:
        return 0, 0, 0, 1
    
    # if pred balls contains all gt balls tp
    for gt_ball in gt_balls:
        for pred_ball in pred_balls:
            # get the first ball in the list
            gt_ball = (gt_balls[0]['x'], gt_balls[0]['y'])
            pred_ball = pred_balls[0]
            
            #calculate euclidean distance
            distance = np.linalg.norm(np.array(gt_ball) - np.array(pred_ball))

            if distance < distance_threshold:
                return 1, 0, 0, 0
            
    return 0, 0, 0 ,0

def calculate_agg_metrics(tp, fp, tn, fn):
    # Calculate metrics
    accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    metrics = {'Accuracy': accuracy, 'F1 Score': f1, 'Precision': precision, 'Recall': recall}
    return metrics

In [7]:
def calculate_positioning_error(gt_balls, pred_balls, prev_position = None):
    if len(gt_balls) == 0 and len(pred_balls) == 0:
        return 0, None
    
    if len(gt_balls) == 0 and len(pred_balls) > 0:
        return -1 , pred_balls[0]
    
    if len(gt_balls) > 0 and len(pred_balls) == 0:
        return -1 , None
        
    pred_ball= None
    # if more than one preditcion get the closest one to prefious position
    if len(pred_balls) > 1 and prev_position is not None:
        distances = []
        for pred_ball in pred_balls:
            distance = np.linalg.norm(np.array(prev_position) - np.array(pred_ball))
            distances.append(distance)
        pred_ball = pred_balls[np.argmin(distances)]
    else:
        pred_ball = pred_balls[0]

    # get the first ball in the list
    gt_ball = (gt_balls[0]['x'], gt_balls[0]['y'])
    pred_ball = pred_balls[0]

    #calculate euclidean distance
    distance = np.linalg.norm(np.array(gt_ball) - np.array(pred_ball))

    return distance, pred_ball

def calculate_positioning_error_df(df):
    errors = []
    prev_position = None
    for index, row in df.iterrows():
        gt_balls = row['points']
        pred_balls = row['ball_points']
        error, prev_position = calculate_positioning_error(gt_balls, pred_balls, prev_position)
        errors.append(error)

    # remove -1
    errors = [x for x in errors if x != -1]

    # scale down with factor 2.5
    errors = [x/2.5 for x in errors]

    # count errors in bins
    bins = [0, 1, 2, 3, 4, 5, 10, 1000]

    # create counts for bins
    counts = np.bincount(np.digitize(errors, bins))

    # median
    mean = np.mean(errors)

    # median below th
    filtered_errors = [x for x in errors if x < 4]
    mean_below_4 = np.mean(filtered_errors)

    return counts, mean, mean_below_4

## Evaluation

In [8]:
# read annotation json
with open('../../00_Dataset/annotations_ball.json') as f:
    data_test = json.load(f)

In [9]:
df_train = convert_json_to_df(data_test, split='train')
df_validation = convert_json_to_df(data_test, split='validation')
df_test = convert_json_to_df(data_test, split='test')

In [10]:
def createResults(df, split):
    result_df= pd.DataFrame(columns=['split','subset', 'video', 'clip', 'frame', 'ball_points', 'player_bbox'])
    unique_combinations = df[['subset', 'video', 'clip']].drop_duplicates()

    # loop over unique combinations
    for i in tqdm(range(len(unique_combinations)), desc="Processing videos"):
        filtered_df = get_frames(df, unique_combinations.iloc[i]['subset'], unique_combinations.iloc[i]['video'], unique_combinations.iloc[i]['clip'])
        filtered_df = process_image_sequence_just_annotations(filtered_df)
        result_df = pd.concat([result_df, filtered_df])

    # update split
    result_df['split'] = split

    return result_df

result_train = createResults(df_train, 'train')
result_validation = createResults(df_validation, 'validation')
result_test = createResults(df_test, 'test')

Processing videos: 100%|██████████| 13/13 [00:54<00:00,  4.23s/it]


In [15]:
metrics_strict_val = calculate_metrics_df(result_validation, loose=False)
metrics_loose_val= calculate_metrics_df(result_validation, loose=True)
metrics_strict_test = calculate_metrics_df(result_test, loose=False)
metrics_loose_test = calculate_metrics_df(result_test, loose=True)
metrics_strict_train = calculate_metrics_df(result_train, loose=False)
metrics_loose_train = calculate_metrics_df(result_train, loose=True)

metrics = {'Strict': {'Train': metrics_strict_train, 'Validation': metrics_strict_val, 'Test': metrics_strict_test},
              'Loose': {'Train': metrics_loose_train, 'Validation': metrics_loose_val, 'Test': metrics_loose_test}}

metrics_strict_test

{'Dataset_Amateur': {'tp': 395, 'fp': 1110, 'tn': 134, 'fn': 324},
 'Dataset_TrackNet': {'tp': 402, 'fp': 435, 'tn': 15, 'fn': 62},
 'Visibility_Visible': {'tp': 726, 'fp': 982, 'tn': 0, 'fn': 248},
 'Visibility_Difficult': {'tp': 55, 'fp': 273, 'tn': 0, 'fn': 123},
 'Visibility_Blurry': {'tp': 15, 'fp': 35, 'tn': 0, 'fn': 5},
 'Visibility_Occluded': {'tp': 1, 'fp': 17, 'tn': 0, 'fn': 10},
 'Visibility_None': {'tp': 0, 'fp': 238, 'tn': 149, 'fn': 0},
 'Total': {'tp': 797, 'fp': 1545, 'tn': 149, 'fn': 386}}

In [16]:
elements = []

for type in metrics.keys():
    for split in metrics[type].keys():
        for key, value in metrics[type][split].items():
            tp = value['tp']
            fp = value['fp']
            tn = value['tn']
            fn = value['fn']
            metrics_key = calculate_agg_metrics(tp, fp, tn, fn)

            row = {'type': type, 'split': split, 'subset': key, 'accuracy': metrics_key['Accuracy'], 'f1': metrics_key['F1 Score'], 'precision': metrics_key['Precision'], 'recall': metrics_key['Recall']}
            elements.append(row)

results = pd.DataFrame(elements)
results[results['subset'] == 'Total']

,type,split,subset,accuracy,f1,precision,recall
7,Strict,Train,Total,0.335950,0.481552,0.373059,0.679028
15,Strict,Validation,Total,0.344920,0.493951,0.403373,0.636986
23,Strict,Test,Total,0.328815,0.452199,0.340307,0.673711
31,Loose,Train,Total,0.617802,0.747967,0.832487,0.679028
39,Loose,Validation,Total,0.612619,0.745657,0.899033,0.636986
47,Loose,Test,Total,0.602548,0.718665,0.770048,0.673711


In [17]:
results[results['subset'].str.contains('Dataset', na=False) & (results['split'] == "Test") & (results['type'] == "Strict")]

,type,split,subset,accuracy,f1,precision,recall
16,Strict,Test,Dataset_Amateur,0.269485,0.355216,0.262458,0.549374
17,Strict,Test,Dataset_TrackNet,0.456236,0.617986,0.480287,0.866379


In [18]:
results[results['subset'].str.contains('Visibility', na=False) & (results['split'] == "Test") & (results['type'] == "Strict")]

,type,split,subset,accuracy,f1,precision,recall
18,Strict,Test,Visibility_Visible,0.371166,0.541387,0.425059,0.745380
19,Strict,Test,Visibility_Difficult,0.121951,0.217391,0.167683,0.308989
20,Strict,Test,Visibility_Blurry,0.272727,0.428571,0.300000,0.750000
21,Strict,Test,Visibility_Occluded,0.035714,0.068966,0.055556,0.090909
22,Strict,Test,Visibility_None,0.385013,0.000000,0.000000,0.000000


In [19]:
test_results = results[(results['split'] == "Test")].reset_index(drop=True)
test_results

,type,split,subset,accuracy,f1,precision,recall
0,Strict,Test,Dataset_Amateur,0.269485,0.355216,0.262458,0.549374
1,Strict,Test,Dataset_TrackNet,0.456236,0.617986,0.480287,0.866379
2,Strict,Test,Visibility_Visible,0.371166,0.541387,0.425059,0.745380
3,Strict,Test,Visibility_Difficult,0.121951,0.217391,0.167683,0.308989
4,Strict,Test,Visibility_Blurry,0.272727,0.428571,0.300000,0.750000
5,Strict,Test,Visibility_Occluded,0.035714,0.068966,0.055556,0.090909
6,Strict,Test,Visibility_None,0.385013,0.000000,0.000000,0.000000
7,Strict,Test,Total,0.328815,0.452199,0.340307,0.673711
8,Loose,Test,Dataset_Amateur,0.506220,0.604900,0.672913,0.549374
9,Loose,Test,Dataset_TrackNet,0.794286,0.881579,0.897321,0.866379


In [20]:
test_results = test_results.pivot(index=['type', 'split'], columns='subset', values='f1').reset_index()
test_results

subset,type,split,Dataset_Amateur,Dataset_TrackNet,Total,Visibility_Blurry,Visibility_Difficult,Visibility_None,Visibility_Occluded,Visibility_Visible
0,Loose,Test,0.604900,0.881579,0.718665,0.857143,0.472103,0.0,0.166667,0.854118
1,Strict,Test,0.355216,0.617986,0.452199,0.428571,0.217391,0.0,0.068966,0.541387


In [22]:
errors_train, mean_train, mean_sub_t_train  = calculate_positioning_error_df(result_train)
errors_val, mean_val, mean_sub_t_val = calculate_positioning_error_df(result_validation)
errors_test, mean_test, mean_sub_t_test = calculate_positioning_error_df(result_test)

# print errors
print("Train")
print(errors_train)
print(mean_train)
print(mean_sub_t_train)

print("Val")
print(errors_val)
print(mean_val)
print(mean_sub_t_val)

print("Test")
print(errors_test)
print(mean_test)
print(mean_sub_t_test)

Train
[   0 2668 1032  323   86   48  235 5297]
81.10228135204217
0.912657337754983
Val
[   0  579  212   86   26    8   37 1099]
73.17138703591903
0.9564784326444246
Test
[   0  679  205   49   13    8   29 1270]
83.26392626840533
0.7693430787040763


In [23]:
# add test pe rsults to DF
test_results['Mean PE'] = mean_test
test_results['Mean PE below th'] = mean_sub_t_test
test_results['Positioning_Error_Counts'] = None  # Initialize the column
test_results['Positioning_Error_Counts'] = test_results['Positioning_Error_Counts'].astype(object)
# Use iterrows to set the entire list for each row
for i, row in test_results.iterrows():
    test_results.at[i, 'Positioning_Error_Counts'] = errors_test

test_results

subset,type,split,Dataset_Amateur,Dataset_TrackNet,Total,Visibility_Blurry,Visibility_Difficult,Visibility_None,Visibility_Occluded,Visibility_Visible,Mean PE,Mean PE below th,Positioning_Error_Counts
0,Loose,Test,0.604900,0.881579,0.718665,0.857143,0.472103,0.0,0.166667,0.854118,83.263926,0.769343,"[0, 679, 205, 49, 13, 8, 29, 1270]"
1,Strict,Test,0.355216,0.617986,0.452199,0.428571,0.217391,0.0,0.068966,0.541387,83.263926,0.769343,"[0, 679, 205, 49, 13, 8, 29, 1270]"


## track time

In [53]:
def benchmark(func, *args, repeats=10):
    times = []
    for _ in range(repeats):
        start = time.perf_counter()
        func(*args)
        end = time.perf_counter()
        times.append(end - start)
    avg_time = sum(times) / repeats
    return avg_time
    
average_time = benchmark(createResults, df_test, 'test', repeats=2)
print(f"Average elapsed time: {average_time:.6f} seconds")

fps = len(df_test) / average_time
print("FPS: ", fps)

Processing videos: 100%|██████████| 13/13 [00:42<00:00,  3.25s/it]

Average elapsed time: 40.970923 seconds
FPS:  70.22053255440498


In [56]:
test_results

subset,type,Dataset_New,Dataset_TrackNet,Total,Visibility_Blurry,Visibility_Difficult,Visibility_None,Visibility_Occluded,Visibility_Visible,Mean PE,Mean PE below th,Positioning_Error_Counts,FPS,Precision Total,Recall Total
0,Loose,0.604900,0.881579,0.718665,0.857143,0.472103,0.0,0.166667,0.854118,83.263926,0.769343,"[0, 679, 205, 49, 13, 8, 29, 1270]",70.220533,0.770048,0.673711
1,Strict,0.355216,0.617986,0.452199,0.428571,0.217391,0.0,0.068966,0.541387,83.263926,0.769343,"[0, 679, 205, 49, 13, 8, 29, 1270]",70.220533,0.340307,0.673711


In [61]:
# get precision and recall values
precision_loose = results[(results['split'] == 'Test') & (results['type'] == 'Loose') & (results['subset'] == 'Total')]['precision'].values[0]
recall_loose = results[(results['split'] == 'Test') & (results['type'] == 'Loose') & (results['subset'] == 'Total')]['recall'].values[0]

precision_strict = results[(results['split'] == 'Test') & (results['type'] == 'Strict') & (results['subset'] == 'Total')]['precision'].values[0]
recall_strict = results[(results['split'] == 'Test') & (results['type'] == 'Strict') & (results['subset'] == 'Total')]['recall'].values[0]

# iterate over rows and add
for index, row in test_results.iterrows():
    if row['type'] == 'Loose':
        test_results.at[index, 'Precision Total'] = precision_loose
        test_results.at[index, 'Recall Total'] = recall_loose
    else:
        test_results.at[index, 'Precision Total'] = precision_strict
        test_results.at[index, 'Recall Total'] = recall_strict

# add fps to test_results
test_results['FPS'] = fps

#drop columns split
test_results = test_results.drop(columns=['split'])

# rename type to model
test_results = test_results.rename(columns={'type': 'Model'})
#add prefix to Model
test_results['Model'] = 'Benchmark_' + test_results['Model'].astype(str)

# rename and filter columns
rename_dict = {
    'Model': 'Model',
    'Total': 'F1-Score Total',
    'Precision Total': 'Precision Total',
    'Recall Total': 'Recall Total',
    'Dataset_New': 'F1 Dataset New',
    'Dataset_TrackNet': 'F1 Dataset TrackNet',
    'Visibility_Visible': 'F1 Visibility Visibile',
    'Visibility_Difficult': 'F1 Visibility Difficult',
    'Visibility_Occluded': 'F1 Visibility Occluded',
    'Visibility_Blurry': 'F1 Visibility Blurry',
    'FPS': 'FPS',
    'Mean PE': 'PE Mean',
    'Mean PE below th': 'PE Mean Below Threshold',
    'Positioning_Error_Counts': 'PE Counts',
}

test_results = test_results.rename(columns=rename_dict).reset_index(drop=True)
test_results

subset,Model,F1 Dataset New,F1 Dataset TrackNet,F1-Score Total,F1 Visibility Blurry,F1 Visibility Difficult,Visibility_None,F1 Visibility Occluded,F1 Visibility Visibile,PE Mean,PE Mean Below Threshold,PE Counts,Precision Total,Recall Total,FPS
0,Benchmark_Loose,0.604900,0.881579,0.718665,0.857143,0.472103,0.0,0.166667,0.854118,83.263926,0.769343,"[0, 679, 205, 49, 13, 8, 29, 1270]",0.770048,0.673711,70.220533
1,Benchmark_Strict,0.355216,0.617986,0.452199,0.428571,0.217391,0.0,0.068966,0.541387,83.263926,0.769343,"[0, 679, 205, 49, 13, 8, 29, 1270]",0.340307,0.673711,70.220533


In [ ]:
results.to_csv('../results/Results_CV.csv', index=False)